In [1]:
# Import PadreApp from a package
from padre.app import pypadre
# All interactions with padre should be done via the app

[WARNING] [08:06:31:pypadre - http] Authentication token is NONE. You need to authentication for user mgrani with your current password (or set a new user)


In [2]:
# List all available datasets(Toy Datasets)
print(pypadre.datasets.list())

['Boston_House_Prices', 'Breast_Cancer', 'Diabetes', 'Digits', 'Iris', 'Linnerrud']


In [3]:
# List all currently available local experiments
print(pypadre.experiments.list_experiments())

['Decorated Experiment', 'Test Experiment SVM', 'Test_Incorrect_Execute_Parameters', 'iPython Experiment1 ', 'KNN', 'SVC(Digits)', 'Regression using PCA and SVR(Digits)', 'Gaussian Process Regression', 'Grid search with Random Forest Classifier and PCA(Digits)', 'SVC(Iris)', 'IPython Experiment 3', 'Decision Tree(Iris)', 'Regression using PCA and SVR(Boston_House_Prices)', 'Decision Tree(Digits)', 'PLS Regression(Boston_House_Prices)', 'Grid search with Random Forest Classifier and PCA(Iris)', 'Torch', 'Linear SVR', 'Random Forest Classifier with PCA', 'Experiment using bagging classifier', 'PLS Regression(Diabetes)', 'Test_Incorrect_Estimator', 'KNN Regressor', 'Regression using PCA and SVR(Diabetes)']


In [4]:
# Create a simple experiment via experiment creator utilizing a dataset displayed. Run it with multiple parameters
pypadre.experiment_creator.clear_experiments()
workflow = pypadre.experiment_creator.create_test_pipeline(['isomap embedding'])
pypadre.experiment_creator.create(name='iPython Experiment1 ', 
                                           description='Simple iPython notebook experiment',
                                           workflow=workflow, 
                                           dataset_list=['Digits'])
pypadre.experiment_creator.execute()

# Aimed at new users for running already created experiments or running from JSON files

INFO: 2019-03-08 08:06:31.591 <padre.experimentcreator.ExperimentCreator object at 0x7f5c59976c88>:	iPython Experiment1  created successfully!
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n'
                             '\n'
                             '    Read more in the :ref:`User Guide '
                             '<isomap>`.\n'
                             '\n'
                             '    Parameters\n'
                             '    ----------\n'
                             '    n_neighbors : integer\n'
                             '        number of neighbors to consider for each '
                             'poin

In [5]:
# Using decorators to run experiments
from padre.ds_import import load_sklearn_toys
from padre import *
from padre.app import pypadre
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

@Workflow(exp_name="Decorated Experiment",
          description="Test experiment with decorators and SVC")
def create_test_pipeline():
    estimators = [('SVC', SVC(probability=True))]
    return Pipeline(estimators)


@Dataset(exp_name="Decorated Experiment")
def get_dataset():
    ds = [i for i in load_sklearn_toys()][2]
    return ds



In [6]:
ex = pypadre.experiments.run(decorated=True)
ex = run("Decorated Experiment")  # run the experiment and report
# Aimed at engineers and scientists for quick prototyping and trying out features

creating the workflow
INFO: 2019-03-08 08:06:32.432 Experiment<id:Decorated Experiment>:	start: phase=experiment
INFO: 2019-03-08 08:06:32.433 	Run<id:fce10604-67c7-42b3-a134-b82157ebf44b;name:Decorated Experiment>:	start: phase=run
INFO: 2019-03-08 08:06:32.433 		Split<id:eaabc20b-462d-47f9-b71c-26d9f352fc50;name:Decorated Experiment>:	start: phase=split
INFO: 2019-03-08 08:06:32.889 		Split<id:eaabc20b-462d-47f9-b71c-26d9f352fc50;name:Decorated Experiment>:	stop: phase=split
INFO: 2019-03-08 08:06:32.899 	Run<id:fce10604-67c7-42b3-a134-b82157ebf44b;name:Decorated Experiment>:	stop: phase=run
INFO: 2019-03-08 08:06:32.911 Experiment<id:Decorated Experiment>:	stop: phase=experiment
creating the workflow
INFO: 2019-03-08 08:06:32.965 Experiment<id:Decorated Experiment>:	start: phase=experiment
INFO: 2019-03-08 08:06:32.966 	Run<id:e92b160a-a366-4bf0-bfd3-50f557b1fa21;name:Decorated Experiment>:	start: phase=run
INFO: 2019-03-08 08:06:32.966 		Split<id:276aa832-35eb-4176-bc00-c7e8dbc0dfd

In [7]:
# Creating and running experiments via pure code (example.py)
def create_test_pipeline():
    from sklearn.pipeline import Pipeline
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    # estimators = [('reduce_dim', PCA()), ('clf', SVC())]
    estimators = [('SVC', SVC(probability=True))]
    return Pipeline(estimators)



In [8]:
from padre.ds_import import load_sklearn_toys
from padre.core import Experiment
from padre.base import PadreLogger
from padre.eventhandler import add_logger

logger = PadreLogger()
logger.backend = pypadre.repository
add_logger(logger=logger)
ds = [i for i in load_sklearn_toys()][2]
ex = Experiment(name="Test Experiment SVM",
                    description="Testing Support Vector Machines via SKLearn Pipeline",
                    dataset=ds,
                    workflow=create_test_pipeline(), keep_splits=True, strategy="random")
ex.execute()

INFO: 2019-03-08 08:06:33.501 Experiment<id:Test Experiment SVM>:	start: phase=experiment
INFO: 2019-03-08 08:06:33.502 Experiment<id:Test Experiment SVM>:	start: phase=experiment
INFO: 2019-03-08 08:06:33.503 	Run<id:5ca05eaf-f6c1-4b75-be8b-5dc7ce5ad214;name:Test Experiment SVM>:	start: phase=run
INFO: 2019-03-08 08:06:33.503 	Run<id:5ca05eaf-f6c1-4b75-be8b-5dc7ce5ad214;name:Test Experiment SVM>:	start: phase=run
INFO: 2019-03-08 08:06:33.504 		Split<id:d0aac611-fc9f-4f8e-b254-5f0365dcd800;name:Test Experiment SVM>:	start: phase=split
INFO: 2019-03-08 08:06:33.504 		Split<id:d0aac611-fc9f-4f8e-b254-5f0365dcd800;name:Test Experiment SVM>:	start: phase=split
INFO: 2019-03-08 08:06:33.931 		Split<id:d0aac611-fc9f-4f8e-b254-5f0365dcd800;name:Test Experiment SVM>:	stop: phase=split
INFO: 2019-03-08 08:06:33.942 		Split<id:d0aac611-fc9f-4f8e-b254-5f0365dcd800;name:Test Experiment SVM>:	stop: phase=split
INFO: 2019-03-08 08:06:33.952 	Run<id:5ca05eaf-f6c1-4b75-be8b-5dc7ce5ad214;name:Test Exp

In [9]:
# The PyTorch wrapper supports the use of User Defined Callbacks 
from padre.core.wrappers.wrapper_pytorch import WrapperPytorch, CallBack
class TestCallbacks(CallBack):
    def on_compute_loss(self, loss):
        print('Function on compute loss. Loss = {loss}'.format(loss=loss))

    def on_epoch_end(self, obj):
        print('Epoch ended')

    def on_epoch_start(self, obj):
        print('Epoch started')

    def on_iteration_start(self, obj):
        print('Iteration started')

    def on_iteration_end(self, obj):
        print('Iteration ended')

In [10]:
# Running an experiment with a custom wrapper
from sklearn.pipeline import Pipeline
import os

print(os.getcwd())
# Define the parameters
params = dict()

import json
with open('../proof_of_concept/Pytorch/classification.json') as json_data:
    params = json.load(json_data)

obj = WrapperPytorch(params=params)
obj.set_callbacks([TestCallbacks()])
estimators = [('pytorch', obj)]
workflow = Pipeline(estimators)
ds = [i for i in load_sklearn_toys()][4]

ex = Experiment(name="Torch",
                description="Testing Torch via SKLearn Pipeline",
                dataset=ds,
                workflow=workflow)
ex.execute()

/home/christofer/PycharmProjects/PyPaDRe/tests/examples
INFO: 2019-03-08 08:06:34.192 Experiment<id:Torch>:	start: phase=experiment
INFO: 2019-03-08 08:06:34.193 Experiment<id:Torch>:	start: phase=experiment
INFO: 2019-03-08 08:06:34.194 	Run<id:41ee96f4-e2da-45ff-aadf-2d10af53a371;name:Torch>:	start: phase=run
INFO: 2019-03-08 08:06:34.195 	Run<id:41ee96f4-e2da-45ff-aadf-2d10af53a371;name:Torch>:	start: phase=run
INFO: 2019-03-08 08:06:34.196 		Split<id:a766069c-6647-4285-b140-30d1c787bdd7;name:Torch>:	start: phase=split
INFO: 2019-03-08 08:06:34.196 		Split<id:a766069c-6647-4285-b140-30d1c787bdd7;name:Torch>:	start: phase=split
Epoch started
Iteration started
Function on compute loss. Loss = 0.19405375410957054
1 0.19405375410957054
Iteration ended
Iteration started
Function on compute loss. Loss = 0.2169058789238152
2 0.2169058789238152
Iteration ended
Iteration started
Function on compute loss. Loss = 0.2345075484948347
3 0.2345075484948347
Iteration ended
Iteration started
Functio

/home/christofer/PycharmProjects/test_folder/venv/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [11]:
# Compare the metrics between experiments and also between runs(Displaying pandas dataframe)
from padre.experimentexecutor import ExperimentExecutor
# Create an experiment with multiple runs
params_pca = {'num_components': [2, 3, 4, 5, 6]}
params_svr = {'C': [0.5, 1.0, 1.5],
              'poly_degree': [1, 2, 3]}
params_dict = {'SVR': params_svr, 'pca': params_pca}
workflow = pypadre.experiment_creator.create_test_pipeline(['pca', 'SVR'])
params_dict = pypadre.experiment_creator.convert_alternate_estimator_names(params_dict)
pypadre.experiment_creator.clear_experiments('IPython Experiment 3')
pypadre.experiment_creator.create(name='IPython Experiment 3',
                                  description='Experiment to test the comparison of metrics',
                                  dataset_list=['Boston_House_Prices'],
                                  workflow=workflow,
                                  params=params_dict
                                 )
experiments_executor = ExperimentExecutor(experiments=pypadre.experiment_creator.createExperimentList())
experiments_executor.execute(local_run=True, threads=1)
#from padre.metrics import CompareMetrics
#metrics = CompareMetrics(experiments_list=experiments_executor.experiments)
#print(metrics.show_metrics())
pypadre.metrics_evaluator.add_experiments(experiments_executor.experiments)
print(pypadre.metrics_evaluator.show_metrics())

INFO: 2019-03-08 08:06:34.327 <padre.experimentcreator.ExperimentCreator object at 0x7f5c59976c88>:	IPython Experiment 3 created successfully!
INFO: 2019-03-08 08:06:34.327 <padre.experimentcreator.ExperimentCreator object at 0x7f5c59976c88>:	IPython Experiment 3 created successfully!
Sequential Execution
INFO: 2019-03-08 08:06:34.402 <padre.experimentexecutor.ExperimentExecutor object at 0x7f5c54538358>:	Executing experiment: iPython Experiment1 
INFO: 2019-03-08 08:06:34.402 <padre.experimentexecutor.ExperimentExecutor object at 0x7f5c54538358>:	Executing experiment: iPython Experiment1 
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n

INFO: 2019-03-08 08:06:35.191 		Split<id:37d20a02-d38b-4b6c-b6d2-81c4120a906c;name:iPython Experiment1 >:	stop: phase=split
INFO: 2019-03-08 08:06:35.192 		Split<id:37d20a02-d38b-4b6c-b6d2-81c4120a906c;name:iPython Experiment1 >:	stop: phase=split
INFO: 2019-03-08 08:06:35.192 	Run<id:37c85667-5ed0-495d-b1a6-913b1f1306f2;name:iPython Experiment1 >:	stop: phase=run
INFO: 2019-03-08 08:06:35.192 	Run<id:37c85667-5ed0-495d-b1a6-913b1f1306f2;name:iPython Experiment1 >:	stop: phase=run
INFO: 2019-03-08 08:06:35.192 Experiment<id:iPython Experiment1 >:	stop: phase=experiment
INFO: 2019-03-08 08:06:35.193 Experiment<id:iPython Experiment1 >:	stop: phase=experiment
INFO: 2019-03-08 08:06:35.193 <padre.experimentexecutor.ExperimentExecutor object at 0x7f5c54538358>:	Completed experiment: iPython Experiment1  with execution time: 0.7904658317565918
INFO: 2019-03-08 08:06:35.193 <padre.experimentexecutor.ExperimentExecutor object at 0x7f5c54538358>:	Completed experiment: iPython Experiment1  with

INFO: 2019-03-08 08:06:35.394 	Run<id:5454fb42-82eb-457e-bf7c-ff3c304402e0;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-03-08 08:06:35.395 	Run<id:5454fb42-82eb-457e-bf7c-ff3c304402e0;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-03-08 08:06:35.395 		Split<id:21726801-dd09-49d7-93b8-e04edab1729c;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-03-08 08:06:35.396 		Split<id:21726801-dd09-49d7-93b8-e04edab1729c;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-03-08 08:06:35.408 		Split<id:21726801-dd09-49d7-93b8-e04edab1729c;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-03-08 08:06:35.409 		Split<id:21726801-dd09-49d7-93b8-e04edab1729c;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-03-08 08:06:35.409 	Run<id:5454fb42-82eb-457e-bf7c-ff3c304402e0;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-03-08 08:06:35.409 	Run<id:5454fb42-82eb-457e-bf7c-ff3c304402e0;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-03-08 08:06:3

INFO: 2019-03-08 08:06:35.601 		Split<id:c2058727-d243-4142-aa79-295704ed4db5;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-03-08 08:06:35.601 		Split<id:c2058727-d243-4142-aa79-295704ed4db5;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-03-08 08:06:35.602 	Run<id:34886972-1e21-4cc0-82c7-fe69e04af690;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-03-08 08:06:35.602 	Run<id:34886972-1e21-4cc0-82c7-fe69e04af690;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-03-08 08:06:35.603 	Run<id:2c7ae2eb-6835-4e17-a9d5-17653925b5e9;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-03-08 08:06:35.604 	Run<id:2c7ae2eb-6835-4e17-a9d5-17653925b5e9;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-03-08 08:06:35.605 		Split<id:b58a9771-bf79-4b5e-b20f-89d471b693b7;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-03-08 08:06:35.605 		Split<id:b58a9771-bf79-4b5e-b20f-89d471b693b7;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-03-08 08:06:3

INFO: 2019-03-08 08:06:35.810 		Split<id:5a0f16e2-5d26-4e84-99f7-c5d44857770c;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-03-08 08:06:35.811 		Split<id:5a0f16e2-5d26-4e84-99f7-c5d44857770c;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-03-08 08:06:35.811 	Run<id:e2bc5730-6cc3-44ec-ad1e-8702d6f26d2c;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-03-08 08:06:35.811 	Run<id:e2bc5730-6cc3-44ec-ad1e-8702d6f26d2c;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-03-08 08:06:35.812 	Run<id:ae29c746-891d-494e-a86a-ff27481914dd;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-03-08 08:06:35.813 	Run<id:ae29c746-891d-494e-a86a-ff27481914dd;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-03-08 08:06:35.814 		Split<id:2f3eba74-7428-4c33-830f-c89a7e7a468a;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-03-08 08:06:35.814 		Split<id:2f3eba74-7428-4c33-830f-c89a7e7a468a;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-03-08 08:06:3

INFO: 2019-03-08 08:06:36.021 		Split<id:c1147fe3-9bda-40fe-b7d5-7a7195997789;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-03-08 08:06:36.021 		Split<id:c1147fe3-9bda-40fe-b7d5-7a7195997789;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-03-08 08:06:36.022 	Run<id:95ce5574-42aa-4b4f-810c-f72c1b873635;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-03-08 08:06:36.022 	Run<id:95ce5574-42aa-4b4f-810c-f72c1b873635;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-03-08 08:06:36.023 	Run<id:11b7ab01-fe7f-4f00-bee4-cf1682f37551;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-03-08 08:06:36.024 	Run<id:11b7ab01-fe7f-4f00-bee4-cf1682f37551;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-03-08 08:06:36.025 		Split<id:4b81cc76-a4c5-4e8d-aa74-a0d1fee1f908;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-03-08 08:06:36.025 		Split<id:4b81cc76-a4c5-4e8d-aa74-a0d1fee1f908;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-03-08 08:06:3

In [12]:
# Analyze metrics where the runs have specific values for a component
pypadre.metrics_evaluator.analyze_runs(
    ['principal component analysis.num_components.4', 'principal component analysis.num_components.5'])
df = pypadre.metrics_evaluator.display_results()
print(df)

                                     run  \
0   34f53a21-2a77-4453-8437-adec20f9d66e   
1   11b7ab01-fe7f-4f00-bee4-cf1682f37551   
2   80312fa8-884a-40f7-9d29-488040c56900   
3   5454fb42-82eb-457e-bf7c-ff3c304402e0   
4   71bc76cb-13d2-4bbd-8e55-95d911fa0d49   
5   a159c127-6c4b-4c82-81b4-53050a4c0fb3   
6   8a6ea3aa-1eca-45e9-93eb-7fe064f9f147   
7   14ff5dce-f251-4556-9f49-9966950a417f   
8   f6492696-e912-4ce6-864a-6868095f5f61   
9   70ef31f8-68da-4a9d-8528-f4b09d37748c   
10  cc6d1a94-da76-4dac-9a6b-66c82c137241   
11  e2bc5730-6cc3-44ec-ad1e-8702d6f26d2c   
12  c8659d1a-a7c7-471b-a75c-cfd30b954790   
13  802e3828-88b6-4eb7-b4c0-4df40e038f2a   
14  8a83c3cc-3119-432b-ae5f-1eba496db50b   
15  00a1592f-06e3-4eab-a4f4-98b161c5af3d   
16  a10f9457-117e-4f9d-8a14-8a02ba39c18b   
17  93203d56-c21e-4e68-914e-ec641db01950   

                                   split                      dataset  \
0   805097ba-bc0a-40f9-97ef-32360379bc58  Boston House Prices dataset   
1   4b81cc76-a4c5

In [13]:
# Display only the metric specified by the user
pypadre.metrics_evaluator.analyze_runs(['principal component analysis.num_components.4'], ['mean_error'])
df = pypadre.metrics_evaluator.display_results()
print(df)

                                    run                                 split  \
0  34f53a21-2a77-4453-8437-adec20f9d66e  805097ba-bc0a-40f9-97ef-32360379bc58   
1  11b7ab01-fe7f-4f00-bee4-cf1682f37551  4b81cc76-a4c5-4e8d-aa74-a0d1fee1f908   
2  71bc76cb-13d2-4bbd-8e55-95d911fa0d49  7d7ac658-8e84-4f80-ae19-e107edad5e4b   
3  8a6ea3aa-1eca-45e9-93eb-7fe064f9f147  46265fef-d045-4bd2-9e52-4256daa9d307   
4  14ff5dce-f251-4556-9f49-9966950a417f  9b9bd575-0adf-4098-9e1e-00652178ccbc   
5  f6492696-e912-4ce6-864a-6868095f5f61  c482b327-7004-462f-ac1e-9b91a127443d   
6  70ef31f8-68da-4a9d-8528-f4b09d37748c  feb6727e-52cd-4e58-a1b1-b2dc52bce132   
7  802e3828-88b6-4eb7-b4c0-4df40e038f2a  a17f03d8-d438-4672-ba44-6e87cdccb600   
8  93203d56-c21e-4e68-914e-ec641db01950  bf407eac-6b2b-4c3a-bcdc-71620fabe3dc   

                       dataset epsilon-support vector regression.poly_degree  \
0  Boston House Prices dataset                                             2   
1  Boston House Prices datase

In [14]:
# Go online, register new user
# For registering, user has to use the browser to enter details

In [15]:
# Log in to server
# Implemented

In [16]:
# Upload/Sync experiments on demand
# Not Implemented. Experiments can only be uploaded during the execution of the experiment

In [17]:
# Look at the experiment via SPA and check the results and visualize them(if possible)

In [18]:
# Download an experiment from the server and run it.(via the experiment.JSON file) Example for reproducibility
# Not yet implemented

In [19]:
# Keep track of random seeds used in the experiment
# Not yet implemented

In [20]:
# Download a dataset from OpenML and upload it to the server
# Generate metrics for uploaded  datasets
# Currently in progress. Some datasets in OpenML do not work and raise exceptions

In [21]:
# Download a dataset from server
# Implemented

In [22]:
# Running an experiment with a dataset downloaded from the server and sync the experiment automatically
# Not implemented

In [23]:
# Running an experiment on the server cluster

In [24]:
# Tracking the progress of experiments including hyperparameter search
# Code to be included in the backend for logging the porogress

In [25]:
# Visualizing the results of an experiment/ dataset